Mostly copied from Heng Zheng, simply added PCA and parquet run https://www.kaggle.com/hengzheng/lightgbm-5folds-baseline
I just realized there is a leak, pca should be performed per fold. I'll fix it, please use future versions.


In [ ]:
import warnings
warnings.simplefilter('ignore')

import gc

import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error

import joblib
import lightgbm as lgb
import pickle

In [ ]:
%%time

train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
print(train.shape)
train.head()

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
ycol = 'target'
feature_names = list(filter(lambda x: x not in [ycol, 'row_id', 'time_id', 'investment_id'], train.columns))
pca = PCA(n_components=50)
pca.fit(train[feature_names])

In [ ]:
filename = 'pca.sav'
pickle.dump(pca, open(filename, 'wb'))

In [ ]:
train[feature_names[:50]] = pca.transform(train[feature_names])

In [ ]:
train.drop(feature_names[50:], axis=1, inplace = True)

In [ ]:
feature_names = list(filter(lambda x: x not in [ycol, 'row_id', 'time_id'], train.columns))

In [ ]:

params = {'boosting_type': 'dart', 'categorical_column': 0, 'objective':'rmse',
           'learning_rate': 0.1167477918474961,
           'min_child_samples': 34, 'min_child_weight': 24.733795444735957,
           'n_estimators': 182, 'n_jobs': -1, 'num_leaves': 268, 'reg_alpha': 2.479425579240586,
           'reg_lambda': 0.2347046165637776, 'subsample': 0.46059222140633244}
model = lgb.LGBMRegressor(**params)

df_importance_list = []

kfold = KFold(n_splits=5, shuffle=True, random_state=2022)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(train[feature_names], train[ycol])):
    X_train = train.iloc[trn_idx][feature_names]
    Y_train = train.iloc[trn_idx][ycol]
    X_val = train.iloc[val_idx][feature_names]
    Y_val = train.iloc[val_idx][ycol]
    lgb_model = model.fit(X_train,
                          Y_train,
                          eval_names=['train', 'valid'],
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                          verbose=100,
                          eval_metric='rmse',
                          early_stopping_rounds=10)
    joblib.dump(lgb_model, f'lgb_{fold_id}.pkl')
    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)
    del lgb_model, X_train, Y_train, X_val, Y_val
    gc.collect()